# Import Libraries

In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
from lazypredict.Supervised import LazyClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier

# Import Dataset

In [28]:
# fetch dataset 
raw_df = pd.read_csv('dataset/Framingham Dataset.csv')

In [29]:
raw_df

,RANDID,SEX,TOTCHOL,AGE,SYSBP,DIABP,CURSMOKE,CIGPDAY,BMI,DIABETES,...,CVD,HYPERTEN,TIMEAP,TIMEMI,TIMEMIFC,TIMECHD,TIMESTRK,TIMECVD,TIMEDTH,TIMEHYP
0,2448,1,195.0000,39,106.0000,70.0000,0,0.0000,26.9700,0,...,1,0,8766,6438,6438,6438,8766,6438,8766,8766
1,2448,1,209.0000,52,121.0000,66.0000,0,0.0000,NaN,0,...,1,0,8766,6438,6438,6438,8766,6438,8766,8766
2,6238,2,250.0000,46,121.0000,81.0000,0,0.0000,28.7300,0,...,0,0,8766,8766,8766,8766,8766,8766,8766,8766
3,6238,2,260.0000,52,105.0000,69.5000,0,0.0000,29.4300,0,...,0,0,8766,8766,8766,8766,8766,8766,8766,8766
4,6238,2,237.0000,58,108.0000,66.0000,0,0.0000,28.5000,0,...,0,0,8766,8766,8766,8766,8766,8766,8766,8766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11622,9998212,1,173.0000,46,126.0000,82.0000,0,0.0000,19.1700,0,...,0,1,8766,8766,8766,8766,8766,8766,8766,0
11623,9998212,1,153.0000,52,143.0000,89.0000,0,0.0000,25.7400,0,...,0,1,8766,8766,8766,8766,8766,8766,8766,0
11624,9999312,2,196.0000,39,133.0000,86.0000,1,30.0000,20.9100,0,...,0,1,8766,8766,8766,8766,8766,8766,8766,4201
11625,9999312,2,240.0000,46,138.0000,79.0000,1,20.0000,26.3900,0,...,0,1,8766,8766,8766,8766,8766,8766,8766,4201


# EDA

In [30]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11627 entries, 0 to 11626
Data columns (total 39 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   RANDID    11627 non-null  int64  
 1   SEX       11627 non-null  int64  
 2   TOTCHOL   11218 non-null  float64
 3   AGE       11627 non-null  int64  
 4   SYSBP     11627 non-null  float64
 5   DIABP     11627 non-null  float64
 6   CURSMOKE  11627 non-null  int64  
 7   CIGPDAY   11548 non-null  float64
 8   BMI       11575 non-null  float64
 9   DIABETES  11627 non-null  int64  
 10  BPMEDS    11034 non-null  float64
 11  HEARTRTE  11621 non-null  float64
 12  GLUCOSE   10187 non-null  float64
 13  educ      11332 non-null  float64
 14  PREVCHD   11627 non-null  int64  
 15  PREVAP    11627 non-null  int64  
 16  PREVMI    11627 non-null  int64  
 17  PREVSTRK  11627 non-null  int64  
 18  PREVHYP   11627 non-null  int64  
 19  TIME      11627 non-null  int64  
 20  PERIOD    11627 non-null  in

In [31]:
# Initialize empty lists for categorical, numerical, and binary features
categorical = []
numerical = []

# Iterate through columns and append to appropriate list
for column in raw_df.columns:
    if raw_df[column].dtype == 'object':
        categorical.append(column)
    elif raw_df[column].dtype in ['float64', 'int64']:
        numerical.append(column)
        
print('Categorical:', categorical)
print('Numerical:', numerical)


Categorical: []
Numerical: ['RANDID', 'SEX', 'TOTCHOL', 'AGE', 'SYSBP', 'DIABP', 'CURSMOKE', 'CIGPDAY', 'BMI', 'DIABETES', 'BPMEDS', 'HEARTRTE', 'GLUCOSE', 'educ', 'PREVCHD', 'PREVAP', 'PREVMI', 'PREVSTRK', 'PREVHYP', 'TIME', 'PERIOD', 'HDLC', 'LDLC', 'DEATH', 'ANGINA', 'HOSPMI', 'MI_FCHD', 'ANYCHD', 'STROKE', 'CVD', 'HYPERTEN', 'TIMEAP', 'TIMEMI', 'TIMEMIFC', 'TIMECHD', 'TIMESTRK', 'TIMECVD', 'TIMEDTH', 'TIMEHYP']


In [32]:
# # Calculate number of rows and columns for the grid
# num_cols = 3  # Number of columns in the grid
# num_rows_numerical = (len(numerical) + num_cols - 1) // num_cols  # Calculate number of rows needed for numerical features
# num_rows_categorical = (len(categorical) + num_cols - 1) // num_cols  # Calculate number of rows needed for categorical features

# # Plot numerical features in a grid
# plt.figure(figsize=(20, 4 * num_rows_numerical))  # Adjust height based on number of rows

# for i, feature in enumerate(numerical):
#     plt.subplot(num_rows_numerical, num_cols, i + 1)
#     sns.histplot(data=raw_df, x=feature, bins=15, hue='CVD')
#     plt.title(f'Histogram of {feature}')
#     plt.xlabel(feature)
#     plt.ylabel('Frequency')

# plt.tight_layout()
# plt.show()


# # Plot categorical features in a grid
# plt.figure(figsize=(20, 4 * num_rows_categorical))  # Adjust height based on number of rows

# for i, feature in enumerate(categorical):
#     plt.subplot(num_rows_categorical, num_cols, i + 1)
#     sns.countplot(y=feature, data=raw_df, hue='CVD')
#     plt.title(f'Bar Plot of {feature}')
#     plt.xlabel('Count')
#     plt.ylabel(feature)

# plt.tight_layout()
# plt.show()

In [33]:
# # Get value counts for all columns
# value_counts = {column: raw_df[column].value_counts() for column in raw_df.columns}

# # Print value counts for all columns
# for column, counts in value_counts.items():
#     print(f"Value counts for column '{column}':")
#     print(counts)
#     print()

# Preprocessing
## Handling Missing Values

In [34]:
# counter percentage missing value in dataset (Function)
def missing_value(data_frame):
    total = data_frame.isnull().sum().sort_values(ascending=False)
    percentage = (data_frame.isnull().sum() / data_frame.isnull().count() * 100).sort_values(ascending=False)
    dtypes = data_frame.dtypes
    return pd.concat([total, percentage, dtypes], axis=1, keys=['Total', 'Percentage', 'Dtypes'])

In [35]:
missing_value(raw_df)

,Total,Percentage,Dtypes
LDLC,8601,73.9744,float64
HDLC,8600,73.9658,float64
GLUCOSE,1440,12.3850,float64
BPMEDS,593,5.1002,float64
TOTCHOL,409,3.5177,float64
educ,295,2.5372,float64
CIGPDAY,79,0.6795,float64
BMI,52,0.4472,float64
HEARTRTE,6,0.0516,float64
TIMEAP,0,0.0000,int64


In [36]:
def impute_median(data):
    return data.fillna(data.median())

raw_df['TOTCHOL'] = raw_df['TOTCHOL'].transform(impute_median)
raw_df['CIGPDAY'] = raw_df['CIGPDAY'].transform(impute_median)
raw_df['BMI'] = raw_df['BMI'].transform(impute_median)
raw_df['BPMEDS'] = raw_df['BPMEDS'].transform(impute_median)
raw_df['HEARTRTE'] = raw_df['HEARTRTE'].transform(impute_median)
raw_df['GLUCOSE'] = raw_df['GLUCOSE'].transform(impute_median)
raw_df['educ'] = raw_df['educ'].transform(impute_median)

In [37]:
missing_value(raw_df)

,Total,Percentage,Dtypes
LDLC,8601,73.9744,float64
HDLC,8600,73.9658,float64
CVD,0,0.0000,int64
DEATH,0,0.0000,int64
ANGINA,0,0.0000,int64
HOSPMI,0,0.0000,int64
MI_FCHD,0,0.0000,int64
ANYCHD,0,0.0000,int64
STROKE,0,0.0000,int64
RANDID,0,0.0000,int64


## Outliers

In [38]:
# # Assuming `raw_df` is your dataframe and `numerical` is a list of numerical column names
# fig, axes = plt.subplots(nrows=num_rows_numerical, ncols=num_cols, figsize=(15, 5 * num_rows_numerical))

# # Flatten the axes array to make it easier to iterate over
# axes = axes.flatten()

# # Create box plots for each selected feature
# for i, feature in enumerate(numerical):
#     sns.boxplot(raw_df[feature], color='blue', ax=axes[i])
#     axes[i].set_title(f'{feature} Distribution')
#     axes[i].set_xlabel(f'Amount Spent on {feature}')
#     axes[i].set_ylabel('Amount')

# # Remove any extra subplots if there are fewer features than subplots
# for j in range(i + 1, num_rows_numerical * num_cols):
#     fig.delaxes(axes[j])

# # Adjust layout
# plt.tight_layout()
# plt.show()


In [39]:
raw_df = raw_df[raw_df['CIGPDAY'] <= 40]
raw_df = raw_df[(raw_df['DIABP'] > 40) & (raw_df['DIABP'] < 140)]
raw_df = raw_df[(raw_df['BMI'] <= 50) & (raw_df['BMI'] >= 20)]
raw_df = raw_df[raw_df['HEARTRTE'] <= 200]
raw_df = raw_df[raw_df['GLUCOSE'] < 450]
raw_df = raw_df[raw_df['TOTCHOL'] < 500]

In [40]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# import numpy as np

# # Assuming `raw_df` is your dataframe and `numerical` is a list of numerical column names
# fig, axes = plt.subplots(nrows=num_rows_numerical, ncols=num_cols, figsize=(15, 5 * num_rows_numerical))

# # Flatten the axes array to make it easier to iterate over
# axes = axes.flatten()

# # Create box plots for each selected feature
# for i, feature in enumerate(numerical):
#     sns.boxplot(raw_df[feature], color='blue', ax=axes[i])
#     axes[i].set_title(f'{feature} Distribution')
#     axes[i].set_xlabel(f'Amount Spent on {feature}')
#     axes[i].set_ylabel('Amount')

# # Remove any extra subplots if there are fewer features than subplots
# for j in range(i + 1, num_rows_numerical * num_cols):
#     fig.delaxes(axes[j])

# # Adjust layout
# plt.tight_layout()
# plt.show()


## Drop Absurd data

In [41]:
raw_df.drop(columns=['LDLC', 'HDLC', 'RANDID'], inplace=True)

In [42]:
filtered_df = raw_df[(raw_df['STROKE'] == 0) & (raw_df['CVD'] == 1) ]

indices_to_drop = filtered_df.index
raw_df.drop(indices_to_drop, inplace=True)

In [43]:
raw_df.drop(columns=['TIMEHYP', 'TIMEDTH', 'TIMESTRK', 'TIMECHD', 'TIMEMI', 'TIME', 'TIMEAP','TIMECVD', 'TIMEMIFC', 'PERIOD'], inplace=True)

In [44]:
raw_df.drop(columns=['CVD','ANGINA', 'ANYCHD', 'HOSPMI', 'MI_FCHD', 'DEATH'], inplace=True)

# Features Correlations

In [45]:
# # Calculate correlations with 'Class_Mapped'
# correlations = raw_df.corr()['CVD'].drop('CVD')

# # Plotting the correlations with explicit hue assignment
# plt.figure(figsize=(10, 6))
# sns.barplot(x=correlations.index, y=correlations.values, hue=correlations.index, dodge=False, palette='viridis', legend=False)

# plt.xlabel('Features')
# plt.ylabel('Correlation with Class_Mapped')
# plt.title('Correlation between Features and Class_Mapped')
# plt.xticks(rotation=45)
# plt.grid(True, axis='y', linestyle='--', alpha=0.7)

# plt.tight_layout()
# plt.show()

In [46]:
raw_df

,SEX,TOTCHOL,AGE,SYSBP,DIABP,CURSMOKE,CIGPDAY,BMI,DIABETES,BPMEDS,HEARTRTE,GLUCOSE,educ,PREVCHD,PREVAP,PREVMI,PREVSTRK,PREVHYP,STROKE,HYPERTEN
2,2,250.0000,46,121.0000,81.0000,0,0.0000,28.7300,0,0.0000,95.0000,76.0000,2.0000,0,0,0,0,0,0,0
3,2,260.0000,52,105.0000,69.5000,0,0.0000,29.4300,0,0.0000,80.0000,86.0000,2.0000,0,0,0,0,0,0,0
4,2,237.0000,58,108.0000,66.0000,0,0.0000,28.5000,0,0.0000,80.0000,71.0000,2.0000,0,0,0,0,0,0,0
5,1,245.0000,48,127.5000,80.0000,1,20.0000,25.3400,0,0.0000,75.0000,70.0000,1.0000,0,0,0,0,0,0,0
6,1,283.0000,54,141.0000,89.0000,1,30.0000,25.3400,0,0.0000,75.0000,87.0000,1.0000,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11621,1,185.0000,40,141.0000,98.0000,0,0.0000,25.6000,0,0.0000,67.0000,72.0000,3.0000,0,0,0,0,1,0,1
11623,1,153.0000,52,143.0000,89.0000,0,0.0000,25.7400,0,0.0000,65.0000,72.0000,3.0000,0,0,0,0,1,0,1
11624,2,196.0000,39,133.0000,86.0000,1,30.0000,20.9100,0,0.0000,85.0000,80.0000,3.0000,0,0,0,0,0,0,1
11625,2,240.0000,46,138.0000,79.0000,1,20.0000,26.3900,0,0.0000,90.0000,83.0000,3.0000,0,0,0,0,0,0,1


# Split

In [47]:
def split_data(X, y, test_size, random_state=42):
    return train_test_split(X, y, test_size=test_size, random_state=random_state)

# Imbalance

In [48]:
from imblearn.over_sampling import SMOTE

def oversample(df):
    X_sample = df.drop(columns=['STROKE'])
    y_sample = df['STROKE']
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X_sample, y_sample)
    oversampled_df = pd.DataFrame(X_resampled, columns=X_sample.columns)
    oversampled_df['STROKE'] = y_resampled
    return oversampled_df.sample(frac=1, random_state=42).reset_index(drop=True)

def undersample(df):
    minority_class = df[df['STROKE'] == 1]
    majority_class = df[df['STROKE'] == 0]
    
    # Determine the size of the minority class
    n_minority = len(minority_class)
    
    # Sample from the majority class to match the size of the minority class
    majority_class_undersampled = majority_class.sample(n=n_minority, random_state=42)
    
    # Concatenate minority and undersampled majority classes
    undersampled_df = pd.concat([minority_class, majority_class_undersampled])
    
    return undersampled_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Standardize

In [49]:
def scale_data(X_train, X_test):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    return X_train_scaled, X_test_scaled

# Classifier

In [50]:
def train_classifier(classifier, X_train, y_train):
    if classifier == 'dt':
        clf = DecisionTreeClassifier()
    elif classifier == 'rf':
        clf = RandomForestClassifier()
    elif classifier == 'svm':
        clf = SVC()
    elif classifier == 'knn':
        clf = KNeighborsClassifier()
    elif classifier == 'xgb':
        clf = XGBClassifier()
    else:
        raise ValueError(f"Classifier '{classifier}' not supported.")
    
    clf.fit(X_train, y_train)
    return clf

In [51]:
def run_experiment(df, balance_method, test_size, classifier):
    if balance_method == 'oversample':
        df = oversample(df)
    elif balance_method == 'undersample':
        df = undersample(df)
    
    X = df.drop(columns=['STROKE'])
    y = df['STROKE']
    
    # Split the data
    X_train, X_test, y_train, y_test = split_data(X, y, test_size)
    
    # Scale the data
    X_train, X_test = scale_data(X_train, X_test)
    
     # Train classifier based on the classifier parameter
    clf = train_classifier(classifier, X_train, y_train)
    
    y_pred = clf.predict(X_test)
    
    # Get the classification report as a dict
    report = classification_report(y_test, y_pred, output_dict=True)
    conf_matrix = confusion_matrix(y_test, y_pred)
    
    return report, conf_matrix

In [54]:
raw_df

,SEX,TOTCHOL,AGE,SYSBP,DIABP,CURSMOKE,CIGPDAY,BMI,DIABETES,BPMEDS,HEARTRTE,GLUCOSE,educ,PREVCHD,PREVAP,PREVMI,PREVSTRK,PREVHYP,STROKE,HYPERTEN
2,2,250.0000,46,121.0000,81.0000,0,0.0000,28.7300,0,0.0000,95.0000,76.0000,2.0000,0,0,0,0,0,0,0
3,2,260.0000,52,105.0000,69.5000,0,0.0000,29.4300,0,0.0000,80.0000,86.0000,2.0000,0,0,0,0,0,0,0
4,2,237.0000,58,108.0000,66.0000,0,0.0000,28.5000,0,0.0000,80.0000,71.0000,2.0000,0,0,0,0,0,0,0
5,1,245.0000,48,127.5000,80.0000,1,20.0000,25.3400,0,0.0000,75.0000,70.0000,1.0000,0,0,0,0,0,0,0
6,1,283.0000,54,141.0000,89.0000,1,30.0000,25.3400,0,0.0000,75.0000,87.0000,1.0000,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11621,1,185.0000,40,141.0000,98.0000,0,0.0000,25.6000,0,0.0000,67.0000,72.0000,3.0000,0,0,0,0,1,0,1
11623,1,153.0000,52,143.0000,89.0000,0,0.0000,25.7400,0,0.0000,65.0000,72.0000,3.0000,0,0,0,0,1,0,1
11624,2,196.0000,39,133.0000,86.0000,1,30.0000,20.9100,0,0.0000,85.0000,80.0000,3.0000,0,0,0,0,0,0,1
11625,2,240.0000,46,138.0000,79.0000,1,20.0000,26.3900,0,0.0000,90.0000,83.0000,3.0000,0,0,0,0,0,0,1


In [52]:
# Define the scenarios
scenarios = [
   ('dt','imbalance', 0.1),
    ('dt','imbalance', 0.2),
    ('dt','imbalance', 0.3),
    ('dt','oversample', 0.1),
    ('dt','oversample', 0.2),
    ('dt','oversample', 0.3),
    ('dt','undersample', 0.1),
    ('dt','undersample', 0.2),
    ('dt','undersample', 0.3),
    ('rf','imbalance', 0.1),
    ('rf','imbalance', 0.2),
    ('rf','imbalance', 0.3),
    ('rf','oversample', 0.1),
    ('rf','oversample', 0.2),
    ('rf','oversample', 0.3),
    ('rf','undersample', 0.1),
    ('rf','undersample', 0.2),
    ('rf','undersample', 0.3),
    ('knn','imbalance', 0.1),
    ('knn','imbalance', 0.2),
    ('knn','imbalance', 0.3),
    ('knn','oversample', 0.1),
    ('knn','oversample', 0.2),
    ('knn','oversample', 0.3),
    ('knn','undersample', 0.1),
    ('knn','undersample', 0.2),
    ('knn','undersample', 0.3),
    ('xgb','imbalance', 0.1),
    ('xgb','imbalance', 0.2),
    ('xgb','imbalance', 0.3),
    ('xgb','oversample', 0.1),
    ('xgb','oversample', 0.2),
    ('xgb','oversample', 0.3),
    ('xgb','undersample', 0.1),
    ('xgb','undersample', 0.2),
    ('xgb','undersample', 0.3),
    ('svm','imbalance', 0.1),
    ('svm','imbalance', 0.2),
    ('svm','imbalance', 0.3),
    ('svm','oversample', 0.1),
    ('svm','oversample', 0.2),
    ('svm','oversample', 0.3),
    ('svm','undersample', 0.1),
    ('svm','undersample', 0.2),
    ('svm','undersample', 0.3),
]

# Create an empty dataframe to store the results
results_df = pd.DataFrame()

# Loop through each scenario
for classifier, balance_method, test_size in scenarios:
    # Run the experiment
    report, conf_matrix = run_experiment(raw_df, balance_method, test_size, classifier)
    
    # Flatten the classification report dict
    report_flat = {}

     # Add the scenario details
    report_flat['scenario'] = f'{classifier}_{balance_method}_split_{int(test_size*100)}%'

    for key_outer, value_outer in report.items():
        if isinstance(value_outer, dict):
            for key_inner, value_inner in value_outer.items():
                report_flat[f'{key_outer}-{key_inner}'] = value_inner
        else:
            report_flat[key_outer] = value_outer
    
   
    
    # Append the result to the dataframe
    # results_df = results_df.append(report_flat, ignore_index=True)
    results_df = pd.concat([results_df, pd.DataFrame([report_flat])], ignore_index=True)


    # Print and plot the confusion matrix
    # print(f"\nConfusion Matrix for {balance_method} with test size {test_size*100}%:")
    # print(conf_matrix)
    # plt.figure(figsize=(8, 6))
    # sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False,
    #             xticklabels=['No CVD', 'CVD'], yticklabels=['No CVD', 'CVD'])
    # plt.xlabel('Predicted')
    # plt.ylabel('Actual')
    # plt.title(f'Confusion Matrix: {balance_method} - Test Size {int(test_size*100)}%')
    # plt.show()
    
# Set precision for float display
pd.options.display.float_format = '{:.4f}'.format

desired_columns = [
    'scenario', 'accuracy',
    '0-precision', '0-recall', '0-f1-score', '0-support',
    '1-precision', '1-recall', '1-f1-score', '1-support',
    'macro avg-precision', 'macro avg-recall', 'macro avg-f1-score', 'macro avg-support',
    'weighted avg-precision', 'weighted avg-recall', 'weighted avg-f1-score', 'weighted avg-support'
]

# Reorder columns in results_df
results_df = results_df[desired_columns]

results_df


,scenario,accuracy,0-precision,0-recall,0-f1-score,0-support,1-precision,1-recall,1-f1-score,1-support,macro avg-precision,macro avg-recall,macro avg-f1-score,macro avg-support,weighted avg-precision,weighted avg-recall,weighted avg-f1-score,weighted avg-support
0,dt_imbalance_split_10%,0.8498,0.9297,0.9030,0.9162,835,0.2500,0.3214,0.2813,84,0.5899,0.6122,0.5987,919,0.8676,0.8498,0.8581,919
1,dt_imbalance_split_20%,0.8471,0.9283,0.8998,0.9138,1656,0.2876,0.3681,0.3229,182,0.6080,0.6339,0.6184,1838,0.8649,0.8471,0.8553,1838
2,dt_imbalance_split_30%,0.8534,0.9192,0.9170,0.9181,2470,0.2979,0.3042,0.3010,286,0.6086,0.6106,0.6096,2756,0.8548,0.8534,0.8541,2756
3,dt_oversample_split_10%,0.8632,0.8619,0.8510,0.8564,785,0.8643,0.8744,0.8693,852,0.8631,0.8627,0.8629,1637,0.8632,0.8632,0.8631,1637
4,dt_oversample_split_20%,0.8653,0.8693,0.8570,0.8631,1622,0.8614,0.8734,0.8674,1651,0.8654,0.8652,0.8652,3273,0.8653,0.8653,0.8652,3273
5,dt_oversample_split_30%,0.8611,0.8760,0.8402,0.8577,2447,0.8474,0.8819,0.8643,2463,0.8617,0.8610,0.8610,4910,0.8617,0.8611,0.8610,4910
6,dt_undersample_split_10%,0.6667,0.6800,0.6602,0.6700,103,0.6535,0.6735,0.6633,98,0.6667,0.6668,0.6666,201,0.6671,0.6667,0.6667,201
7,dt_undersample_split_20%,0.6915,0.6588,0.7277,0.6915,191,0.7277,0.6588,0.6915,211,0.6933,0.6933,0.6915,402,0.6950,0.6915,0.6915,402
8,dt_undersample_split_30%,0.6584,0.6293,0.6560,0.6424,282,0.6861,0.6604,0.6730,321,0.6577,0.6582,0.6577,603,0.6595,0.6584,0.6587,603
9,rf_imbalance_split_10%,0.9227,0.9263,0.9940,0.9590,835,0.7826,0.2143,0.3364,84,0.8545,0.6041,0.6477,919,0.9132,0.9227,0.9021,919
